In [1]:
import os
import sys
import torch
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../voice_cloning')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../voice_cloning/Grad-TTS')))
#sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '/repo/voice-cloning/voice_cloning/BigVGAN')))
from speaker_encoder import ecapa_tdnn
from model import tts, unit_encoder
from text.symbols import symbols
import params
from tqdm import tqdm

2025-04-11 20:00:18 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


In [2]:
ckpt = torch.load('/repo/voice-cloning/voice_cloning/Grad-TTS/checkpts/pretrained_unit_encoder.pt')
ckpt.keys()

dict_keys(['model'])

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ecapa_state_dict = torch.load('../voice_cloning/Grad-TTS/checkpts/speaker_encoder.pt')
grad_state_dict = torch.load('/repo/voice-cloning/voice_cloning/Grad-TTS/logs/unit_encoder/gradtts_58.pt')
unit_state_dict = torch.load('/repo/voice-cloning/voice_cloning/Grad-TTS/logs/unit_encoder_3/unit_encoder_128.pt')

ecapa = ecapa_tdnn.ECAPA_TDNN_SMALL_WTTH_PROJ()
add_blank = params.add_blank
nsymbols = len(symbols) + 1 if add_blank else len(symbols)
n_spks = params.n_spks
spk_emb_dim = params.spk_emb_dim
n_enc_channels = params.n_enc_channels
filter_channels = params.filter_channels
filter_channels_dp = params.filter_channels_dp
n_enc_layers = params.n_enc_layers
enc_kernel = params.enc_kernel
enc_dropout = params.enc_dropout
n_heads = params.n_heads
window_size = params.window_size

n_feats = params.n_feats
n_fft = params.n_fft
sample_rate = params.sample_rate
hop_length = params.hop_length
win_length = params.win_length
f_min = params.f_min
f_max = params.f_max

dec_dim = params.dec_dim
beta_min = params.beta_min
beta_max = params.beta_max
pe_scale = params.pe_scale

n_units = params.n_units
grad_tts_model = tts.GradTTS(nsymbols, n_spks, spk_emb_dim, n_enc_channels,
                    filter_channels, filter_channels_dp,
                    n_heads, n_enc_layers, enc_kernel, enc_dropout, window_size,
                    n_feats, dec_dim, beta_min, beta_max, pe_scale)
model = unit_tts.UnitGradTTS(n_units, n_spks, spk_emb_dim, n_enc_channels,
                    filter_channels, filter_channels_dp,
                    n_heads, n_enc_layers, enc_kernel, enc_dropout, window_size,
                    n_feats, dec_dim, beta_min, beta_max, pe_scale).cuda()
    
# model.load_state_dict(torch.load('../voice_cloning/Grad-TTS/checkpts/gradtts_40.pt', map_location=lambda loc, storage: loc))
grad_tts_model.load_state_dict(grad_state_dict)
model.decoder = grad_tts_model.decoder
model.encoder = unit_encoder.UnitEncoder(n_units, n_feats, n_enc_channels, filter_channels,
                    filter_channels_dp, n_heads, n_enc_layers, enc_kernel,
                    enc_dropout, window_size, n_contentvec=0).cuda()
model.encoder.load_state_dict(unit_state_dict)
model.encoder.requires_grad = False
ecapa.load_state_dict(ecapa_state_dict)
ecapa.eval().cuda()



/repo/voice-cloning/venv/lib/python3.12/site-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (256) may be set too high. Or, the value for `n_freqs` (257) may be set too low.
  warnings.warn(


ECAPA_TDNN_SMALL_WTTH_PROJ(
  (spk): ECAPA_TDNN(
    (feature_extract): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
    (instance_norm): InstanceNorm1d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (layer1): Conv1dReluBn(
      (conv): Conv1d(256, 512, kernel_size=(5,), stride=(1,), padding=(2,))
      (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (layer2): SE_Res2Block(
      (Conv1dReluBn1): Conv1dReluBn(
        (conv): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
        (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (Res2Conv1dReluBn): Res2Conv1dReluBn(
        (convs): ModuleList(
          (0-6): 7 x Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        )
        (bns): ModuleList(
          (0-6): 7 x BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stat

In [4]:
# model.spk_emb = ecapa
ecapa.requires_grad = False
model.encoder.requires_grad = False
model.to(device)

UnitGradTTS(
  (spk_emb): Embedding(247, 64)
  (encoder): UnitEncoder(
    (emb): Embedding(1000, 192)
    (prenet): ConvReluNorm(
      (conv_layers): ModuleList(
        (0-2): 3 x Conv1d(192, 192, kernel_size=(5,), stride=(1,), padding=(2,))
      )
      (norm_layers): ModuleList(
        (0-2): 3 x LayerNorm()
      )
      (relu_drop): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.5, inplace=False)
      )
      (proj): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
    )
    (encoder): Encoder(
      (drop): Dropout(p=0.1, inplace=False)
      (attn_layers): ModuleList(
        (0-5): 6 x MultiHeadAttention(
          (conv_q): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (conv_k): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (conv_v): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (conv_o): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (drop): Dropout(p=0.1, inplace=False)
        )
      )
      (norm_layers_1): Modu

In [5]:
import torchaudio
import librosa
import IPython.display as ipd

In [6]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../voice_cloning/Grad-TTS/hifi-gan')))

In [7]:
from pathlib import Path
# Add project root to path
if str(Path.cwd().parent) not in sys.path:
    sys.path.append(str(Path.cwd().parent))
sys.path.append("../voice_cloning/textlesslib")

from voice_cloning.textlesslib.textless.data.speech_encoder import SpeechEncoder


In [8]:
model.spk_emb = ecapa
model.spk_emb.requires_grad = False
model.encoder.requires_grad = False
model.to(device)

2025-04-11 20:00:21 | INFO | fairseq.tasks.hubert_pretraining | current directory is /repo/voice-cloning/notebooks
2025-04-11 20:00:21 | INFO | fairseq.tasks.hubert_pretraining | HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': '/checkpoint/annl/s2st/data/voxpopuli/mHuBERT/en_es_fr', 'fine_tuning': False, 'labels': ['km'], 'label_dir': '/checkpoint/wnhsu/experiments/hubert/kmeans/mhubert_vp_en_es_fr_it2_400k/en_es_fr.layer9.km500', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
2025-04-11 20:00:21 | INFO | fairseq.models.hubert.hubert | HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_mode': default, 'encoder_layers': 12, 'encoder_embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_attention_heads': 12, 'activation_fn': gelu, 'layer_type': transformer, 'dropout'

In [9]:

_ = unit_extractor.cuda().eval()
unit_extractor.requires_grad_(False);

In [10]:

def process_unit(encoded, sampling_rate, hop_length):
    # A method that aligns units and durations (50Hz) extracted from 16kHz audio with
    # mel-spectrograms extracted from 22,050Hz audio.

    unit = encoded["units"].cpu().tolist()
    duration = encoded["durations"].cpu().tolist()

    duration = [int(i) * (sampling_rate // 50) for i in duration]

    expand_unit = []

    for u, d in zip(unit, duration):
        for _ in range(d):
            expand_unit.append(u)

    new_length = len(expand_unit) // hop_length * hop_length

    unit = torch.LongTensor(expand_unit)[:new_length].reshape(-1, hop_length).mode(1)[0].tolist()

    squeezed_unit = [unit[0]]
    squeezed_duration = [1]

    for u in unit[1:]:
        if u == squeezed_unit[-1]:
            squeezed_duration[-1] += 1
        else:
            squeezed_unit.append(u)
            squeezed_duration.append(1)

    unit = torch.LongTensor(squeezed_unit)
    duration = torch.LongTensor(squeezed_duration)

    return unit, duration

In [11]:
source_path = "Amir2.wav"
source_path = "Nikita_sample.wav"
wav, sr = librosa.load(source_path)
print('Source speech')
ipd.display(ipd.Audio(wav, rate=sr))
wav = torch.FloatTensor(wav).unsqueeze(0)

Source speech


OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec

In [5]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../voice_cloning/Grad-TTS/hifi-gan')))

from data import UnitMelSpeakerDataset, UnitMelSpeakerBatchCollate
from torch.utils.data import DataLoader
train_dataset = UnitMelSpeakerDataset('finetune.txt', '../voice_cloning/Grad-TTS/resources/cmu_dictionary', params.add_blank,
                                          params.n_fft, params.n_feats, params.sample_rate, params.hop_length,
                                          params.win_length, params.f_min, params.f_max)
batch_collate = UnitMelSpeakerBatchCollate()
loader = DataLoader(dataset=train_dataset, batch_size=1,
                        collate_fn=batch_collate, drop_last=True,
                        num_workers=1, shuffle=True)

In [6]:
resample_fn = torchaudio.transforms.Resample(sr, params.sample_rate)
spk = resample_fn(wav)
# torchaudio.save("Amir2.wav", spk, 24000)
spk = ecapa(spk)
orig_sr = sr

torch.Size([1, 490177])

In [12]:
#torchaudio.save("Amir.wav", wav, 16000)

In [13]:
sys.path.append("../voice_cloning/Grad-TTS/hifi-gan")
from meldataset import mel_spectrogram

mel = mel_spectrogram(wav, n_fft, n_feats, sample_rate, hop_length,
                              win_length, f_min, f_max, center=False).squeeze()
        

/repo/voice-cloning/venv/lib/python3.12/site-packages/torch/functional.py:709: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at /pytorch/aten/src/ATen/native/SpectralOps.cpp:873.)
  return _VF.stft(  # type: ignore[attr-defined]


In [14]:
model.train()
unit, unit_lengths = unit.unsqueeze(0).cuda(), torch.LongTensor([unit.shape[-1]]).cuda()
duration = duration.unsqueeze(0).cuda()
y, y_lengths = mel.unsqueeze(0).cuda(), torch.LongTensor([mel.shape[-1]]).cuda()
spk = spk.cuda()

In [15]:
optimizer = torch.optim.Adam(params=model.decoder.parameters(), lr=2e-5)
iterations = 800

In [ ]:

model.train()
for iter in tqdm(range(iterations)):
    for batch in loader:
        model.zero_grad()
        x, x_lengths = batch['x'].cuda(), batch['x_lengths'].cuda()
        y, y_lengths = batch['y'].cuda(), batch['y_lengths'].cuda()
        dur_loss, prior_loss, diff_loss = model.compute_loss(x, x_lengths,
                                                                                        y, y_lengths,
                                                                    spk=spk.cuda(), out_size=params.out_size)
        loss = sum([dur_loss, prior_loss, diff_loss])
        loss.backward()
        optimizer.step()

 54%|█████▎    | 429/800 [00:41<00:34, 10.66it/s]

In [ ]:
source_path = "/repo/voice-cloning/notebooks/Amir2.wav"
wav, sr = librosa.load(source_path)
print('Source speech')
ipd.display(ipd.Audio(wav, rate=sr))
wav = torch.FloatTensor(wav).unsqueeze(0)
spk2 = resample_fn(wav)
spk2.shape

Source speech


torch.Size([1, 127200])

In [ ]:
#torchaudio.save("woman.wav", wav, 16000)

In [ ]:
encoded2 = unit_extractor(wav)
unit2, duration2 = process_unit(encoded2, sr, 256)
unit2, unit_lengths2 = unit2.unsqueeze(0).cuda(), torch.LongTensor([unit2.shape[-1]]).cuda()
duration2 = duration2.unsqueeze(0).cuda()

OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.


In [1]:
y_enc, y_dec, attn = model(unit, unit_lengths, duration, spk=spk2, n_timesteps=torch.tensor(50).cuda())

NameError: name 'model' is not defined

In [2]:
from models import Generator as HiFiGAN
from env import AttrDict
import json
with open('/repo/voice-cloning/voice_cloning/Grad-TTS/checkpts/hifigan-config.json') as f:
    h = AttrDict(json.load(f))
hifigan = HiFiGAN(h)
hifigan.load_state_dict(torch.load('/repo/voice-cloning/voice_cloning/Grad-TTS/checkpts/hifigan.pt', map_location=lambda loc, storage: loc)['generator'])
_ = hifigan.cuda().eval()
hifigan.remove_weight_norm()

ModuleNotFoundError: No module named 'models'

/repo/voice-cloning/venv/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [3]:

with torch.no_grad():
    audio = hifigan.forward(y_dec).cpu().squeeze().clamp(-1, 1)
ipd.display(ipd.Audio(audio, rate=22050))

NameError: name 'torch' is not defined

In [4]:
#torchaudio.save("Amir_woman.wav", audio.unsqueeze(0), 22050)

In [ ]:
from model.text_encoder import TextEncoder
from collections import OrderedDict

grad_tts_state_dict = torch.load('/repo/voice-cloning/voice_cloning/Grad-TTS/logs/unit_encoder/gradtts_58.pt')
text_encoder_state_dict = OrderedDict()
for key, value in grad_tts_state_dict.items():
    if "encoder." in key:
        new_key = key.replace("encoder.", "", 1)
        text_encoder_state_dict[new_key] = value

decoder = model.decoder
text_encoder = TextEncoder(nsymbols, n_feats, n_enc_channels, filter_channels,
                    filter_channels_dp, n_heads, n_enc_layers, enc_kernel,
                    enc_dropout, window_size).to(device)
text_encoder.load_state_dict(text_encoder_state_dict)

tts_model = tts.GradTTS(nsymbols, n_spks, spk_emb_dim, n_enc_channels,
                    filter_channels, filter_channels_dp,
                    n_heads, n_enc_layers, enc_kernel, enc_dropout, window_size,
                    n_feats, dec_dim, beta_min, beta_max, pe_scale).to(device)
tts_model.decoder = decoder
tts_model.encoder = text_encoder
tts_model.eval()

GradTTS(
  (spk_emb): Embedding(247, 64)
  (encoder): TextEncoder(
    (emb): Embedding(149, 192)
    (prenet): ConvReluNorm(
      (conv_layers): ModuleList(
        (0-2): 3 x Conv1d(192, 192, kernel_size=(5,), stride=(1,), padding=(2,))
      )
      (norm_layers): ModuleList(
        (0-2): 3 x LayerNorm()
      )
      (relu_drop): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.5, inplace=False)
      )
      (proj): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
    )
    (encoder): Encoder(
      (drop): Dropout(p=0.1, inplace=False)
      (attn_layers): ModuleList(
        (0-5): 6 x MultiHeadAttention(
          (conv_q): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (conv_k): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (conv_v): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (conv_o): Conv1d(192, 192, kernel_size=(1,), stride=(1,))
          (drop): Dropout(p=0.1, inplace=False)
        )
      )
      (norm_layers_1): ModuleLis

In [ ]:
from data import TextMelSpeakerDataset
test_dataset_text = TextMelSpeakerDataset('test.txt', '../voice_cloning/Grad-TTS/resources/cmu_dictionary', params.add_blank,
                                          params.n_fft, params.n_feats, params.sample_rate, params.hop_length,
                                          params.win_length, params.f_min, params.f_max)
x = test_dataset_text[0]['x'].unsqueeze(0)
x_lengths = torch.LongTensor([x.shape[-1]]).cuda()
x.shape, x_lengths

(torch.Size([1, 97]), tensor([97], device='cuda:0'))

In [ ]:
y_enc, y_dec, attn = model(x.cuda(), x_lengths.cuda(), torch.tensor(50).cuda(), spk=spk.cuda())

In [ ]:
with torch.no_grad():
    audio = hifigan.forward(y_dec).cpu().squeeze().clamp(-1, 1)
ipd.display(ipd.Audio(audio, rate=orig_sr))